In [ ]:
import os
lib_folder = os.path.abspath("../src/bin/Release/net10.0/win-x64/publish")

import pythonnet
pythonnet.load("coreclr", runtime_config=os.path.join(lib_folder, "DerivaSharp.runtimeconfig.json"))

import clr
clr.AddReference(os.path.join(lib_folder, "DerivaSharp.dll"))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from DerivaSharp.Instruments import EuropeanOption, OptionType
from DerivaSharp.Models import BsmModelParameters
from DerivaSharp.PricingEngines import AnalyticEuropeanEngine, PricingContext
from System import DateOnly

In [ ]:
option_type = OptionType.Call
strike = 100.0
startDate = DateOnly(2025, 1, 6)
endDate = startDate.AddDays(90)
vol = 0.3
r = 0.04
q = 0.01

option = EuropeanOption(option_type, strike, startDate, endDate)
model = BsmModelParameters(vol, r, q)
engine = AnalyticEuropeanEngine()

In [ ]:
data = []
spots = np.arange(50, 150, 1, dtype=np.float64)
for spot in spots:
    context = PricingContext[BsmModelParameters](model, spot, startDate)
    result = engine.ValueAndGreeks(option, context)
    metrics = {
        "value": result.Value,
        "delta": result.Delta,
        "gamma": result.Gamma,
        "speed": result.Speed,
        "theta": result.Theta,
        "charm": result.Charm,
        "color": result.Color,
        "vega": result.Vega,
        "vanna": result.Vanna,
        "zomma": result.Zomma,
        "rho": result.Rho,
    }
    metrics["spot"] = spot
    data.append(metrics)
df = pd.DataFrame(data)

In [ ]:
greek_columns = [col for col in df.columns if col != "spot"]
n_cols = 3
n_rows = (len(greek_columns) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 3.5 * n_rows))
axes_flat = axes.flatten()

for ax, col in zip(axes_flat, greek_columns):
    ax.plot(df["spot"], df[col], linewidth=2, color="#0052cc")

    if df["spot"].min() <= strike <= df["spot"].max():
        ax.axvline(strike, color="red", linestyle=":", alpha=0.6, label="Strike")

    ax.set_title(col.upper(), fontsize=10, fontweight="bold")
    ax.set_xlabel("Spot Price", fontsize=9)
    ax.grid(True, linestyle="--", alpha=0.5)
    ax.tick_params(labelsize=8)

for i in range(len(greek_columns), len(axes_flat)):
    axes_flat[i].set_visible(False)

plt.tight_layout()
plt.show()